In [2]:
import pandas as pd
file_paths = [f'/root/KRX_LLM/data/fixed_krx_sample_mcqa_data{i}.csv' for i in range(1, 11)]
combined_df = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
combined_df.to_csv('/root/KRX_LLM/data/fixed_krx_sample_mcqa_data.csv', index=False)

In [21]:
import re

def format_text(text):
    # 질문과 선택지 분리
    parts = re.split(r'###\s*선택지:', text)
    
    questions = parts[0]
    
    # 질문에서 불필요한 부분 제거 및 정리
    questions = re.sub(r'###\s*질문:\s*', '', questions)
    questions = re.findall(r"'([^']*)'", questions)
    
    # 실제 질문만 추출 (선택지 제외)
    actual_questions = [q for q in questions if not q.startswith('1.') and not q.startswith('2.') and not q.startswith('3.') and not q.startswith('4.') and not q.startswith('5.')]
    
    # 선택지 추출
    choices = [q for q in questions if q.startswith('1.') or q.startswith('2.') or q.startswith('3.') or q.startswith('4.') or q.startswith('5.')]
    
    # 포맷팅
    formatted_text = "### 질문: \n" + "\n".join(actual_questions) + "\n### 선택지:\n" + "\n".join(choices)
    
    return formatted_text

In [22]:
combined_df['question'] = combined_df['question'].apply(format_text)
combined_df

,question,answer
0,"### 질문: \nAI 시스템이 금융 서비스에서 의사결정을 내릴 때, 윤리적 고려사...",A
1,### 질문: \n은행에서 고객 맞춤형 서비스 제공을 위해 어떤 기술의 도입이 필수...,A
2,### 질문: \nAI를 사용하여 보험사가 고객 경험을 향상시키는 방법과 이러한 향...,A
3,### 질문: \n어떤 기술이나 방법이 금융 서비스 접근성을 높이는 데 가장 적합한...,C
4,"### 질문: \nAI 기반 신용 위험 평가 시스템의 장점을 고려할 때, 마이크로파...",C
...,...,...
4944,### 질문: \n미국 회사가 중국 공급자로부터 신용 거래를 통해 제품을 구매하기 ...,B
4945,### 질문: \nJPMorgan Chase와 OnDeck Capital의 협력이 ...,A
4946,### 질문: \nABC Inc.가 DEF GmbH로부터 기계를 수입하기 위해 신용...,C
4947,### 질문: \n상업 은행이 신용 위험을 평가하기 위해 사용하는 회계 기준과 감사...,B


In [26]:
import re

def convert_numbers_to_letters(text):
    # 질문과 선택지 분리
    parts = text.split("### 선택지:")
    question = parts[0].strip()
    choices = parts[1].strip() if len(parts) > 1 else ""

    # 선택지 번호를 알파벳으로 변환
    def replace_number(match):
        number = match.group(1)
        return chr(64 + int(number)) + '.' # A는 65, B는 66, ...

    # 1. 2. 3. 4. 5. 또는 1) 2) 3) 4) 5) 패턴 변환
    choices = re.sub(r'^(\d+)[.)]', replace_number, choices, flags=re.MULTILINE)

    # 선택지 내용 중 숫자만 있는 경우 (1 2 3 4 5) 변환
    choices = re.sub(r'\b(\d+)\b', replace_number, choices)

    return f"{question}\n### 선택지:\n{choices}"

In [27]:
combined_df['question'] = combined_df['question'].apply(convert_numbers_to_letters)
combined_df

,question,answer
0,"### 질문: \nAI 시스템이 금융 서비스에서 의사결정을 내릴 때, 윤리적 고려사...",A
1,### 질문: \n은행에서 고객 맞춤형 서비스 제공을 위해 어떤 기술의 도입이 필수...,A
2,### 질문: \nAI를 사용하여 보험사가 고객 경험을 향상시키는 방법과 이러한 향...,A
3,### 질문: \n어떤 기술이나 방법이 금융 서비스 접근성을 높이는 데 가장 적합한...,C
4,"### 질문: \nAI 기반 신용 위험 평가 시스템의 장점을 고려할 때, 마이크로파...",C
...,...,...
4944,### 질문: \n미국 회사가 중국 공급자로부터 신용 거래를 통해 제품을 구매하기 ...,B
4945,### 질문: \nJPMorgan Chase와 OnDeck Capital의 협력이 ...,A
4946,### 질문: \nABC Inc.가 DEF GmbH로부터 기계를 수입하기 위해 신용...,C
4947,### 질문: \n상업 은행이 신용 위험을 평가하기 위해 사용하는 회계 기준과 감사...,B


In [28]:
combined_df.to_csv('/root/KRX_LLM/data/fixed_mcqa_combined.csv', index=False)